In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mbee import compute_edits_and_insertions
from insert_convert import InsertionConverter, get_number_of_masks
from trying import PointingConverter

In [3]:
import pandas as pd

In [4]:
max_mask = 4
use_pointing = True

In [5]:
label_map = {'PAD': 0, 'SWAP': 1, 'KEEP': 2, 'DELETE': 3}
# Create Insert 1 MASK to insertion N MASKS.
for i in range(1, max_mask+1):
    label_map[f'KEEP|{i}'] = len(label_map)
if not use_pointing:
    label_map[f'DELETE|{i}'] = len(label_map)

In [6]:
label_map

{'PAD': 0,
 'SWAP': 1,
 'KEEP': 2,
 'DELETE': 3,
 'KEEP|1': 4,
 'KEEP|2': 5,
 'KEEP|3': 6,
 'KEEP|4': 7}

In [7]:
compute_edits_and_insertions(['haryo', 'makan', 'jokowi'], ['haryo',  'nasi', 'jokowi', 'goreng', 'makan', 'enak'], 5)

(['[KEEP]', '[KEEP]', '[DELETE]'],
 [['nasi', 'jokowi', 'goreng'], [], ['enak']])

In [8]:
point_converter = PointingConverter({}, False)

In [9]:
points = point_converter.compute_points(['[CLS]', 'haryo', 'makan', '[SEP]'], ' '.join(['[CLS]', 'haryo',  'nasi', 'jokowi', 'goreng', 'makan', 'enak','[SEP]']))

In [10]:
points

[1|, 2|nasi jokowi goreng, 3|enak, 0|]

In [11]:
points = point_converter.compute_points(
    ["CLS", "a", "b", "a", "##d", "##e", "[SEP]"],
    " ".join(["[CLS]", "a", "a", "##d", "##e", "[SEP]"]),
)

In [12]:
points

[1|, 3|, 0|, 4|, 5|, 6|, 0|]

In [13]:
labels = [t.added_phrase for t in points]
point_indexes = [t.point_index for t in points]
point_indexes_set = set(point_indexes)
new_labels = []
for i, added_phrase in enumerate(labels):
    if i not in point_indexes_set:
        new_labels.append(label_map['DELETE'])
    elif not added_phrase:
        new_labels.append(label_map['KEEP'])
    else:
        new_labels.append(label_map['KEEP|' +
                                            str(len(added_phrase.split()))])
labels = new_labels

In [14]:
point_indexes_set

{0, 1, 3, 4, 5, 6}

In [15]:
labels

[2, 2, 3, 2, 2, 2, 2]

In [16]:
insertion_converter = InsertionConverter(20, label_map, tokenized_data, use_pointing)

NameError: name 'tokenized_data' is not defined

In [17]:
from transformers import AutoTokenizer, AutoModel

/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AutoTokenizer.from_pretrained('bert-base-cased')

In [18]:
tokenizer_bert = AutoTokenizer.from_pretrained('indolem/indobert-base-uncased')

In [19]:
def create_label_dict(max_mask=3, use_pointing=True):
    label_map = {'PAD': 0, 'SWAP': 1, 'KEEP': 2, 'DELETE': 3}
    # Create Insert 1 MASK to insertion N MASKS.
    for i in range(1, max_mask+1):
        label_map[f'KEEP|{i}'] = len(label_map)
    if not use_pointing:
        label_map[f'DELETE|{i}'] = len(label_map)
    return label_map

In [20]:
label_dict = create_label_dict()

In [21]:
import pandas as pd

In [22]:
df_train = pd.read_csv("../data/stif-indonesia/train.csv")

In [31]:

# regex of xxxWORDxxx
df_train[df_train.formal.str.contains("xxx.*xxx")]

# replace such words with xWORDx
df_train.formal = df_train.formal.str.replace("xxx", "x")
df_train.informal = df_train.informal.str.replace("xxx", "x")

In [33]:
example_instance = df_train.iloc[0]

In [34]:
mbek = compute_edits_and_insertions(example_instance.informal, example_instance.formal, 5)

In [35]:
informal_instance = tokenizer_bert.tokenize(example_instance.informal, add_special_tokens=True)
formal_instance = tokenizer_bert.tokenize(example_instance.formal, add_special_tokens=True)

In [36]:
point_converter = PointingConverter({}, False)

In [37]:
points = point_converter.compute_points(informal_instance, ' '.join(formal_instance))

In [38]:
def create_pointer_labels(points, label_map):
    labels = [t.added_phrase for t in points]
    point_indexes = [t.point_index for t in points]
    point_indexes_set = set(point_indexes)
    new_labels = []
    for i, added_phrase in enumerate(labels):
        if i not in point_indexes_set:
            new_labels.append(label_map["DELETE"])
        elif not added_phrase:
            new_labels.append(label_map["KEEP"])
        else:
            new_labels.append(label_map["KEEP|" + str(len(added_phrase.split()))])
    return new_labels

In [39]:
label_dict

{'PAD': 0,
 'SWAP': 1,
 'KEEP': 2,
 'DELETE': 3,
 'KEEP|1': 4,
 'KEEP|2': 5,
 'KEEP|3': 6}

In [40]:
label = create_pointer_labels(points, label_dict)

In [41]:
point_indexes = [t.point_index for t in points]

In [42]:
def get_pointer_and_label(x, label_dict, point_converter):
    informal_instance = tokenizer_bert.tokenize(x.informal, add_special_tokens=True)
    formal_instance = tokenizer_bert.tokenize(x.formal, add_special_tokens=True)
    points = point_converter.compute_points(informal_instance, ' '.join(formal_instance))
    label = create_pointer_labels(points, label_dict)
    point_indexes = [t.point_index for t in points]
    return point_indexes, label
    

In [43]:
label_dict = create_label_dict(30)


In [44]:
label_dict

{'PAD': 0,
 'SWAP': 1,
 'KEEP': 2,
 'DELETE': 3,
 'KEEP|1': 4,
 'KEEP|2': 5,
 'KEEP|3': 6,
 'KEEP|4': 7,
 'KEEP|5': 8,
 'KEEP|6': 9,
 'KEEP|7': 10,
 'KEEP|8': 11,
 'KEEP|9': 12,
 'KEEP|10': 13,
 'KEEP|11': 14,
 'KEEP|12': 15,
 'KEEP|13': 16,
 'KEEP|14': 17,
 'KEEP|15': 18,
 'KEEP|16': 19,
 'KEEP|17': 20,
 'KEEP|18': 21,
 'KEEP|19': 22,
 'KEEP|20': 23,
 'KEEP|21': 24,
 'KEEP|22': 25,
 'KEEP|23': 26,
 'KEEP|24': 27,
 'KEEP|25': 28,
 'KEEP|26': 29,
 'KEEP|27': 30,
 'KEEP|28': 31,
 'KEEP|29': 32,
 'KEEP|30': 33}

In [45]:
a = df_train.apply(
    lambda x: get_pointer_and_label(x, label_dict, point_converter), axis=1
)
# unpack to two series
point_indexes, label = zip(*a)

# append to dataframe
df_train['point_indexes'] = point_indexes
df_train['label'] = label
df_train.to_csv("train_with_pointing.csv", index=False)


In [48]:
df_test

,informal,formal
0,kalian juga tdk banyak membantu terkait dengan...,kalian juga tidak banyak membantu terkait deng...
1,"kan akun lu private , jd kaga bisa liat mereka","kan akun kamu private , jadi mereka tidak bisa..."
2,min kenapa akun saya tidak ditemukan ya ? apak...,"admin , mengapa akun saya tidak ditemukan ? ap..."
3,"akun sy sdh premium , tolong cashback yg sehar...","akun saya sudah premium , tolong cashback yang..."
4,tolong cek dm yaa min . thx !,tolong periksa dm min . terima kasih !
...,...,...
358,NaN,NaN
359,NaN,NaN
360,NaN,NaN
361,NaN,NaN


In [51]:
# do it for validation and test too
df_val = pd.read_csv("../data/stif-indonesia/dev.csv")
df_test = pd.read_csv("../data/stif-indonesia/test.csv")

# replace such words with xWORDx
df_val.formal = df_val.formal.str.replace("xxx", "x")
df_val.informal = df_val.informal.str.replace("xxx", "x")

# replace such words with xWORDx
df_test.formal = df_test.formal.str.replace("xxx", "x")
df_test.informal = df_test.informal.str.replace("xxx", "x")


a = df_val.apply(
    lambda x: get_pointer_and_label(x, label_dict, point_converter), axis=1
)
# unpack to two series
point_indexes, label = zip(*a)
df_val['point_indexes'] = point_indexes
df_val['label'] = label

df_val.to_csv("dev_with_pointing.csv", index=False)

a = df_test.apply(
    lambda x: get_pointer_and_label(x, label_dict, point_converter), axis=1
)
# unpack to two series
point_indexes, label = zip(*a)
df_test['point_indexes'] = point_indexes
df_test['label'] = label

df_test.to_csv("test_with_pointing.csv", index=False)

In [52]:
x = df_train.label.tolist()

# flatten x
x = [item for sublist in x for item in sublist]


In [53]:
# count x
from collections import Counter
Counter(x)

Counter({2: 21447,
         3: 12832,
         4: 4068,
         5: 1387,
         6: 672,
         7: 288,
         8: 125,
         9: 69,
         10: 27,
         11: 16,
         12: 6,
         14: 3,
         13: 2,
         17: 1,
         15: 1,
         16: 1})

In [54]:
# calculate len describe label df_train
df_train['len_label'] = df_train.label.apply(lambda x: len(x))
df_train.len_label.describe()

count    1922.000000
mean       21.303330
std         9.291961
min         7.000000
25%        14.000000
50%        20.000000
75%        27.000000
max        87.000000
Name: len_label, dtype: float64

In [55]:
pd.set_option('max_colwidth', 0)

In [56]:
df_train[df_train.len_label > 90]

,informal,formal,point_indexes,label,len_label


In [57]:
df_train.to_csv("train_with_pointing.csv", index=False)

In [58]:
# input: input_tokens, labels, point_indexes, output_tokens
# input_tokens: tokenized input
# labels: label (MASK|xxx, delete, keep, etc)
# point_indexes: index of the token that is pointed to
# output_tokens: tokenized target

informal_instance

['[CLS]',
 'alhamdulillah',
 'st',
 '##l',
 '##h',
 'libur',
 'x',
 '##num',
 '##ber',
 '##x',
 'hari',
 'on',
 '##bi',
 '##d',
 'lg',
 '##sg',
 'dikasih',
 'order',
 '##an',
 ',',
 'food',
 'lg',
 '.',
 'thanks',
 'xu',
 '##ser',
 '##x',
 'cc',
 '[SEP]']

In [59]:
tokenizer_bert.vocab['[PAD]']

0

In [60]:
def create_masked_source(source_tokens, labels, source_indexes,
                        target_tokens):
    """Realizes source_tokens & adds deleted to source_tokens and target_tokens.

    Args:
        source_tokens: List of source tokens.
        labels: List of label IDs, which correspond to a list of labels (KEEP,
        DELETE, MASK|1, MASK|2...).
        source_indexes: List of next tokens (see pointing converter for more
        details) (ordered by source tokens)
        target_tokens: Optional list of target tokens. Only provided when
        constructing training examples.

    Returns:
        masked_tokens: The source input for the insertion model, including MASK
        tokens and bracketed deleted tokens.
        target_tokens: The target tokens for the insertion model, where mask
        tokens are replaced with the actual token, also includes bracketed
        deleted tokens.
    """
    # well its unused...
    DELETE_SPAN_START = '[UNK]'
    DELETE_SPAN_END = '[PAD]'
    current_index = 0
    masked_tokens = []

    label_map_inverse = {v: k for k, v in label_map.items()}

    kept_tokens = set([0])
    for _ in range(len(source_tokens)):
        current_index = source_indexes[current_index]
        kept_tokens.add(current_index)
        # Token is deleted.
        if current_index == 0:
            break

    current_index = 0
    for _ in range(len(source_tokens)):
        source_token = source_tokens[current_index]
        deleted_tokens = []
        # Looking forward finding all deleted tokens.
        for i in range(current_index + 1, len(source_tokens)):
        ## If not a deleted token.
            if i in kept_tokens:
                break
            deleted_tokens.append(source_tokens[i])

        # Add deleted tokens to masked_tokens and target_tokens.
        masked_tokens.append(source_token)
        # number_of_masks specifies the number MASKED tokens which
        # are added to masked_tokens.
        number_of_masks = get_number_of_masks(
            label_map_inverse[labels[current_index]])
        for _ in range(number_of_masks):
            masked_tokens.append('[MASK]')
        if deleted_tokens:
            masked_tokens_length = len(masked_tokens)
            bracketed_deleted_tokens = ([DELETE_SPAN_START] +
                                    deleted_tokens +
                                        [DELETE_SPAN_END])
            target_tokens = (
                target_tokens[:masked_tokens_length] + bracketed_deleted_tokens +
                target_tokens[masked_tokens_length:])
            masked_tokens += bracketed_deleted_tokens

        current_index = source_indexes[current_index]
        if current_index == 0:
            break
    return masked_tokens, target_tokens

In [61]:
example_instance = df_train.iloc[0]
first_label = df_train.label.tolist()[0]
first_point_indexes = df_train.point_indexes.tolist()[0]
informal_instance = tokenizer_bert.tokenize(example_instance.informal, add_special_tokens=True)
formal_instance = tokenizer_bert.tokenize(example_instance.formal, add_special_tokens=True)

In [62]:
tokenizer_bert.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [63]:
print(' '.join(informal_instance))
print(' '.join(formal_instance))

[CLS] alhamdulillah st ##l ##h libur x ##num ##ber ##x hari on ##bi ##d lg ##sg dikasih order ##an , food lg . thanks xu ##ser ##x cc [SEP]
[CLS] alhamdulillah setelah libur x ##num ##ber ##x hari on ##bi ##d langsung diberi order , makanan lagi . terima kasih xu ##ser ##x cc . [SEP]


In [64]:
masked_tokens, target_tokens = create_masked_source(informal_instance, 
                     first_label, 
                     first_point_indexes, 
                     formal_instance)

In [65]:
import random

In [66]:
if target_tokens and '[MASK]' not in masked_tokens:
      # Generate random MASKs.
    # Don't mask the start or end token.
    indexes = list(range(1, len(masked_tokens) - 1))
    random.shuffle(indexes)
    # Limit MASK to ~10% of the source tokens.
    indexes = indexes[:int(len(masked_tokens) * 0.1)]
    for index in indexes:
        masked_tokens[index] = '[MASK]'

In [67]:
len(masked_tokens)

47

In [68]:
list(zip(masked_tokens, target_tokens))

[('[CLS]', '[CLS]'),
 ('alhamdulillah', 'alhamdulillah'),
 ('[MASK]', 'setelah'),
 ('[UNK]', '[UNK]'),
 ('st', 'st'),
 ('##l', '##l'),
 ('##h', '##h'),
 ('[PAD]', '[PAD]'),
 ('libur', 'libur'),
 ('x', 'x'),
 ('##num', '##num'),
 ('##ber', '##ber'),
 ('##x', '##x'),
 ('hari', 'hari'),
 ('on', 'on'),
 ('##bi', '##bi'),
 ('##d', '##d'),
 ('[MASK]', 'langsung'),
 ('[MASK]', 'diberi'),
 ('[UNK]', '[UNK]'),
 ('lg', 'lg'),
 ('##sg', '##sg'),
 ('dikasih', 'dikasih'),
 ('[PAD]', '[PAD]'),
 ('order', 'order'),
 ('[UNK]', '[UNK]'),
 ('##an', '##an'),
 ('[PAD]', '[PAD]'),
 (',', ','),
 ('[MASK]', 'makanan'),
 ('[MASK]', 'lagi'),
 ('[UNK]', '[UNK]'),
 ('food', 'food'),
 ('lg', 'lg'),
 ('[PAD]', '[PAD]'),
 ('.', '.'),
 ('[MASK]', 'terima'),
 ('[MASK]', 'kasih'),
 ('[UNK]', '[UNK]'),
 ('thanks', 'thanks'),
 ('[PAD]', '[PAD]'),
 ('xu', 'xu'),
 ('##ser', '##ser'),
 ('##x', '##x'),
 ('cc', 'cc'),
 ('[MASK]', '.'),
 ('[SEP]', '[SEP]')]

In [69]:
len(target_tokens)

47